In [2]:
#importazione dei dati ------------------------------------------------------------------------------------------------------------

from pandas import DataFrame
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.linalg import toeplitz


df = pd.read_csv (r'/Users/patrickpinta/Desktop/1datasetpanda/soybean.csv',header=0,error_bad_lines=False) 

#modellazione dei dati training set------------------------------------------------------------------------------------------------------------


x= df.iloc[0:250,0:23]

y= df.iloc[0:250,23:24]

#modellazione dei dati crossvalidation set------------------------------------------------------------------------------------------------------------

xcv=df.iloc[250:334,0:23]

ycv= df.iloc[250:334,23:24]

#stardadizzazione dei dati training set------------------------------------------------------------------------------------------------------------

scaler = MinMaxScaler(feature_range=(-1,1))

x_train_stand=(scaler.fit_transform(x))


#stardadizzazione dei dati crossvalidation set------------------------------------------------------------------------------------------------------------

scaler = MinMaxScaler(feature_range=(-1,1))

xcv_train_stand=(scaler.fit_transform(xcv))
    
#modello dati training set------------------------------------------------------------------------------------------------------------    
    
x_train_stand, y = np.array(x_train_stand), np.array(y)
x_train_stand = sm.add_constant(x_train_stand)

#modello dati training set------------------------------------------------------------------------------------------------------------    

xcv_train_stand, ycv = np.array(xcv_train_stand), np.array(ycv)
xcv_train_stand = sm.add_constant(xcv_train_stand)

#procedura per sigma e per gls model ------------------------------------------------------------------------------------------------------------

ols_resid = sm.OLS(y.astype(float), x_train_stand.astype(float)).fit().resid
resid_fit = sm.OLS(ols_resid[1:], sm.add_constant(ols_resid[:-1])).fit()
rho = resid_fit.params[1]
toeplitz(range(5))
order = toeplitz(range(len(ols_resid)))
sigma = rho**order

gls_model = sm.GLS(y, x_train_stand,sigma)
gls_results = gls_model.fit()

print(gls_results.summary())

#procedura per ols model ------------------------------------------------------------------------------------------------------------

ols_model = sm.OLS(y, x_train_stand)
ols_results = ols_model.fit() 

#print(ols_results.summary())

#identify outliers ------------------------------------------------------------------------------------------------------------
"""
import numpy as np
np.set_printoptions(suppress=True)

influence = ols_results.get_influence()

cooks = influence.cooks_distance

infl = influence.summary_frame().dfb_x1 > 2./250**.5

print(influence.summary_frame()[infl].filter(regex="dfb_x1"))

print(2./417**.5)

"""
#forecast ------------------------------------------------------------------------------------------------------------

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

ypredTrain = gls_results.predict(x_train_stand)
ycvpred = gls_results.predict(xcv_train_stand)

print(r2_score(y,ypredTrain))
print(r2_score(ycv,ycvpred))
print(mean_squared_error(y,ypredTrain))
print(mean_squared_error(ycv,ycvpred))




                            GLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.979
Model:                            GLS   Adj. R-squared:                  0.977
Method:                 Least Squares   F-statistic:                     455.5
Date:                Wed, 12 May 2021   Prob (F-statistic):          1.94e-175
Time:                        11:51:22   Log-Likelihood:                -1188.1
No. Observations:                 250   AIC:                             2424.
Df Residuals:                     226   BIC:                             2509.
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1146.8558     24.689     46.452      0.0